In [120]:
import pandas as pd
import os

sondaje = 'DD3985'
ruta_base = r'C:\Users\house\Desktop\Minera Los Pelambres\Muestrera\Mapeo\Reporte cutting'

ETL Litologia

In [121]:
archivo_litologia = sondaje + '\\litologia.csv'
ruta_litologia = os.path.join(ruta_base, archivo_litologia)
df_litologia = pd.read_csv(ruta_litologia)

#Elimino la columna registro
df_litologia.drop(['Reg'], axis=1, inplace=True)

# Elimino todas las comas de los comentarios
df_litologia['Comentarios'] = df_litologia['Comentarios'].fillna('').astype(str).str.replace(',', '', regex=True)

#Eliminar las filas que contienen sólo 0
df_litologia = df_litologia.dropna(subset=["Desde", "Hasta"], how="all")

df_litologia.to_csv(r'data\\litologia.csv', index=False, header=True)

df_litologia.head(1)

,Desde,Hasta,Litologia,Comentarios
0,0.0,8.0,SI,relleno de plataforma


ETL Volumen de alteracion

In [ ]:
archivo_volalt = sondaje + '\\volalt.csv'
ruta_volalt = os.path.join(ruta_base, archivo_volalt)
df_volalt = pd.read_csv(ruta_volalt)

#Elimino la columna registro
df_volalt.drop(['Reg', 'Litologia'], axis=1, inplace=True)

#Eliminar las filas que contienen sólo 0
df_volalt = df_volalt.dropna(subset=["Desde", "Hasta"], how="all")

# Elimino todas las comas de los comentarios
df_volalt['Comentarios'] = df_volalt['Comentarios'].fillna('').astype(str).str.replace(',', '', regex=True)

# Relleno con 0 las filas sin alteracion
columnas_volalt = ['K (Biot)', 'CS', 'QS', 'SI', 'K (Feld)', 'ARC', 'Epid', 'Cl', 'NR']
df_volalt[columnas_volalt] = df_volalt[columnas_volalt].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

df_volalt.to_csv(r'data\\volalt.csv', index=False, header=True)

df_volalt.head(1)

,Desde,Hasta,K (Biot),CS,QS,SI,K (Feld),ARC,Epid,Cl,NR,K (Hist),Comentarios
0,0.0,8.0,0,0,0,0,0,0,0,0,0,0,Relleno de plataforma


ETL Alteracion

In [123]:
archivo_alteracion = sondaje + '\\alteracion.csv'
ruta_alteracion = os.path.join(ruta_base, archivo_alteracion)
df_alteracion = pd.read_csv(ruta_alteracion)

#Elimino la columna registro
df_alteracion.drop(['Reg', 'Lito'], axis=1, inplace=True)

# Elimino todas las comas de los comentarios
df_alteracion['Comentarios'] = df_alteracion['Comentarios'].fillna('').astype(str).str.replace(',', '', regex=True)

#Eliminar las filas que contienen sólo 0
df_alteracion = df_alteracion.dropna(subset=["Desde", "Hasta"], how="all")

df_alteracion.to_csv(r'data\\alteracion.csv', index=False, header=True)

df_alteracion.head(1)


,Desde,Hasta,Tipo Alt,Comentarios
0,0.0,8.0,SI,Relleno de plataforma


ETL Estructuras

In [124]:
archivo_estructuras = sondaje + '\\estructuras.csv'
ruta_estructuras = os.path.join(ruta_base, archivo_estructuras)
df_estructuras = pd.read_csv(ruta_estructuras)
#Elimino la columna registro
df_estructuras.drop(['Reg'], axis=1, inplace=True)

# Transformo la columna Angulo a valores vacios
df_estructuras['Angulo'] = df_estructuras['Angulo'].fillna('').astype(str)

# Elimino todas las comas de los comentarios
df_estructuras['Comentarios'] = df_estructuras['Comentarios'].fillna('').astype(str).str.replace(',', '', regex=True)

#Eliminar las filas que contienen sólo 0
df_estructuras = df_estructuras.dropna(subset=["Desde", "Hasta"], how="all")

df_estructuras.to_csv(r'data\\estructuras.csv', index=False, header=True)

df_estructuras.head(1)

,Desde,Hasta,Relleno de Est,Angulo,Comentarios
0,0.0,8.0,SI,,Relleno de plataforma


ETL Mineralizacion

In [131]:
archivo_mineralizacion = sondaje + '\\mineralizacion.csv'
ruta_mineralizacion = os.path.join(ruta_base, archivo_mineralizacion)
df_mineralizacion = pd.read_csv(ruta_mineralizacion)
df_mineralizacion.drop(['Ident.', 'Zona mineral'], axis=1, inplace=True)
df_mineralizacion.fillna(0, inplace=True)
#Eliminar filas inválidad, las que tienen todos los valores 0
columnas = ['Desde', 'Hasta', '% Sulf', 'Py', 'Cpy', 'Cc', 'Bo', 'Mo']

#Eliminar las filas que contienen sólo 0
df_mineralizacion = df_mineralizacion.loc[~(df_mineralizacion[columnas] == 0).all(axis=1)]

#Reemplazar las filas cuando tienen valores 0 en las leyes
columnas_leyes = ['% Sulf', 'Py', 'Cpy', 'Cc', 'Bo', 'Mo']
df_mineralizacion.loc[(df_mineralizacion[columnas_leyes] == 0).all(axis=1), ['% Sulf', 'Py']] = 0.001

# Agregar una fila inicial si el primer intervalo no comienza desde 0
if df_mineralizacion['Desde'].iloc[0] > 0:
    first_row = {        
        'Desde': 0,
        'Hasta': df_mineralizacion['Desde'].iloc[0],        
        '% Sulf': 0.001,
        'Py': 0.001,
        'Cpy': 0.0,
        'Cc': 0.0,
        'Bo': 0.0,
        'Mo': 0.0
    }
    # Insertar la fila inicial en la primera posición
    df_mineralizacion = pd.concat([pd.DataFrame([first_row]), df_mineralizacion], ignore_index=True)

# Crear filas para cubrir los gaps entre las filas existentes
rows_to_update = []
for i in range(1, len(df_mineralizacion)):
    prev_hasta = df_mineralizacion.iloc[i - 1]['Hasta']
    curr_desde = df_mineralizacion.iloc[i]['Desde']
    
    if curr_desde > prev_hasta:  # Detectar gap
        gap_row = {            
            'Desde': prev_hasta,
            'Hasta': curr_desde,            
            '% Sulf': 0.001,
            'Py': 0.001,
            'Cpy': 0.0,
            'Cc': 0.0,
            'Bo': 0.0,
            'Mo': 0.0
        }
        rows_to_update.append((i, gap_row))
df_mineralizacion.to_csv(r'data\\mineralizacion.csv', index=False, header=True)
df_mineralizacion.head(1)

,Desde,Hasta,% Sulf,Py,Cpy,Cc,Bo,Mo
0,0.0,16.0,0.001,0.001,0.0,0.0,0.0,0.0


ETL Minzone

In [126]:
archivo_minzone = sondaje + '\\minzone.csv'
ruta_minzone = os.path.join(ruta_base, archivo_minzone)
df_minzone = pd.read_csv(ruta_minzone)
#Elimino la columna registro
df_minzone.drop(['Reg'], axis=1, inplace=True)

# Elimino todas las comas de los comentarios
df_minzone['Comentarios'] = df_minzone['Comentarios'].fillna('').astype(str).str.replace(',', '', regex=True)

#Eliminar las filas que contienen sólo 0
df_minzone = df_minzone.dropna(subset=["Desde", "Hasta"], how="all")

df_minzone.to_csv(r'data\\minzone.csv', index=False, header=True)

df_minzone.head(1)

,Desde,Hasta,MinZone,Comentarios
0,0.0,8.0,SI,Relleno de plataforma


ETL Pisotecho

In [127]:
archivo_pisotecho = sondaje + '\\pisotecho.csv'
ruta_pisotecho = os.path.join(ruta_base, archivo_pisotecho)
df_pisotecho = pd.read_csv(ruta_pisotecho)
#Elimino la columna registro
df_pisotecho.drop(['Reg'], axis=1, inplace=True)

# Elimino todas las comas de los comentarios
df_pisotecho['Comentarios'] = df_pisotecho['Comentarios'].fillna('').astype(str).str.replace(',', '', regex=True)

#Eliminar las filas que contienen sólo 0
df_pisotecho = df_pisotecho.dropna(subset=['Profundidad'], how="all")

df_pisotecho.to_csv(r'data\\pisotecho.csv', index=False, header=True)

df_pisotecho.head(1)

,Profundidad,Piso Techo,Comentarios
0,124.0,6.0,


ETL Vetillas

In [128]:
archivo_vetillas = sondaje + '\\vetillas.csv'
ruta_vetillas = os.path.join(ruta_base, archivo_vetillas)
df_vetillas = pd.read_csv(ruta_vetillas)

#Elimino la columna registro
df_vetillas.drop(['Reg'], axis=1, inplace=True)

# Elimino todas las comas de los comentarios
df_vetillas['Comentarios'] = df_vetillas['Comentarios'].fillna('').astype(str).str.replace(',', '', regex=True)

# Relleno con 0 las filas de las vetillas y comentarios vacíos
columnas_vetillas =['T4', 'A', 'B', 'ED', 'D']
df_vetillas[columnas_vetillas] = df_vetillas[columnas_vetillas].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

#Eliminar las filas que contienen sólo 0
df_vetillas = df_vetillas.dropna(subset=["Desde", "Hasta"], how="all")
df_vetillas.to_csv(r"data\\vetillas.csv", index=False, header=True)
df_vetillas.head(1)

,Desde,Hasta,T4,A,B,ED,D,Comentarios
0,0.0,8.0,0,0,0,0,0,Relleno plataforma


ETL Unidad geologica

In [129]:
archivo_ug = sondaje + '\\UG.csv'
ruta_ug = os.path.join(ruta_base, archivo_ug)
df_ug = pd.read_csv(ruta_ug)
#Elimino la columna registro
df_ug.drop(['Reg'], axis=1, inplace=True)

# Elimino todas las comas de los comentarios
df_ug['Comentarios'] = df_ug['Comentarios'].fillna('').astype(str).str.replace(',', '', regex=True)

#Eliminar las filas que contienen sólo 0
df_ug = df_ug.dropna(subset=["Desde", "Hasta"], how="all")

df_ug.to_csv(r'data\\ug.csv', index=False, header=True)

df_ug.head(1)

,Desde,Hasta,UG,Comentarios
0,0.0,8.0,-9.0,Relleno de plataforma


Pega el siguiente código en  la línea de comandos

In [130]:
# pyinstaller --onefile --hidden-import=comtypes.stream --hidden-import=comtypes --add-data "C:\Python\Mapeo\data\litologia.csv;." --add-data "C:\Python\Mapeo\data\volalt.csv;." --add-data "C:\Python\Mapeo\data\alteracion.csv;." --add-data "C:\Python\Mapeo\data\estructuras.csv;." --add-data "C:\Python\Mapeo\data\mineralizacion.csv;." --add-data "C:\Python\Mapeo\data\minzone.csv;." --add-data "C:\Python\Mapeo\data\pisotecho.csv;." --add-data "C:\Python\Mapeo\data\vetillas.csv;." --add-data "C:\Python\Mapeo\data\ug.csv;." mapeo.py